In [27]:
import re
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from pymongo import MongoClient
from collections.abc import Iterable
import warnings
import time

In [28]:
client = MongoClient()
DB_NAME = 'AA'
db = client[DB_NAME]
elmenus_collection = db['data']

#cur_path = os.getcwd()

In [29]:
def saveToDB(collection, city_areas_zones_dict):
    
    try:
        #print("collection.count_documents({}): ",  collection.count_documents({}))
        resultSet = collection.count_documents({"prim":city_areas_zones_dict["prim"],"jobfunc":city_areas_zones_dict['job_func']})

        if resultSet > 0:
            #print("city_areas_zones_dict is already scrapped >> ", city_areas_zones_dict)
            print("Skip!")
        else:
           # print("New city_areas_zones_dict will be scrapped >> ", city_areas_zones_dict)
            print("I will insert it!")
            collection.insert_one(city_areas_zones_dict)
            
    except Exception as e:
        print('Exception : %s' % str(e))

In [30]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")

In [31]:
def create(url):
    browser = webdriver.Chrome(r'C:\Users\Ali\Downloads\chromedriver_win32\chromedriver.exe')
    browser.get(url)
    return browser

In [32]:
def geter(link):
        bwr = create(link)
        job_title= bwr.find_element_by_class_name('job-title').text
        job_industry=bwr.find_element_by_class_name('industries.labels-wrapper').text
        job_function=bwr.find_element_by_class_name('labels-wrapper').text
        indus=bwr.find_element_by_class_name('job-requirements.content-card')
        job_skills=indus.find_element_by_class_name('labels-wrapper').text
        job_req=indus.find_element_by_tag_name('span').text
        disc=bwr.find_element_by_class_name('about-job.content-card')
        job_disc=disc.find_element_by_tag_name('span').text
        table = bwr.find_element_by_class_name('table').text
        Dict = {'prim':1,'description':job_disc,'req':job_req,'skills':job_skills,'industry':job_industry,'job_func':job_function,'job_title':job_title}
        saveToDB(elmenus_collection, Dict)
        bwr.close()

In [33]:
###intenship page special case###
def internship_handler():
    alllinks=[]
    searchurl='https://wuzzuf.net/a/Internships-in-Egypt?page='
    alljobs=[]
    x=1
    while True:
        temp=create(searchurl+str(x))
        alljobs= temp.find_elements_by_class_name('job-title')
        for i in range(len(alljobs)):
            alllinks.append(alljobs[i].get_attribute('href'))
        temp.close()
        x+=1
        if (x==17):
            break
    return alllinks

In [41]:
def all_related_jobs(link):
    alllinks=[]
    alljobs=[]
    x=0
    try:
        ###all other cases###
        while True:
            temp=create(link+str(x))
            alljobs= temp.find_elements_by_class_name('job-title')
            for i in range(len(alljobs)):
                alllinks.append(alljobs[i].find_element_by_tag_name('a').get_attribute('href'))
            temp.close()
            x+=20
            if(x==500):
                break
    except:
        ###android special case###
        while True:
            android_handler=create(link+str(x))
            alljobs= android_handler.find_elements_by_class_name('job-title')
            for i in range(len(alljobs)):
                alllinks.append(alljobs[i].get_attribute('href'))
            android_handler.close()    
            x+=1
            if (x==8):
                break
        temp.close()           
    return alllinks

In [35]:
def browse_jobs(browser):
    browser.find_element_by_class_name('dropdown.browse-jobs-mega-menu').click()
    list_links=browser.find_elements_by_class_name("job-category-link.truncate")
    uneditedlist=[]
    editedlist=[]
    x='start='
    for i in range(40):
        uneditedlist.append(list_links[i].get_attribute('href'))
    for i in range(len(uneditedlist)):
        editedlist.append(re.sub("ref=browse-jobs.*$", "", uneditedlist[i]))
        editedlist[i]=editedlist[i]+x
    del editedlist[20]   
    return editedlist


In [36]:
def scraper():
    browser=create("https://wuzzuf.net/jobs/egypt")
    time.sleep(2)
    editedlist=browse_jobs(browser)
    all_jobs=[]
    internship_list=internship_handler()
    all_jobs=internship_list
    print(len(all_jobs))
    url=''
    i=0
    for x in range(len(editedlist)):
        url=editedlist[x]
        temp=all_related_jobs(url)
        all_jobs=all_jobs+temp
        print(len(all_jobs))
    for i in range(len(all_jobs)):
        geter(all_jobs[i])    

In [ ]:
scraper()

400
900


Exception ignored in: <function Service.__del__ at 0x0000024AF4C52828>
Traceback (most recent call last):
  File "D:\Applications\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "D:\Applications\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "D:\Applications\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "D:\Applications\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "D:\Applications\Anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "D:\Applications\Anaconda3\lib\socket.py", line 716, in create_connection
    sock.connect(sa)
KeyboardInter